# (OpenCV - Chap6) 화소(pixel)처리
> 영상 화소의 접근과 화소 밝기 변환

- toc: true
- branch: master
- badges: false
- comments: true
- author: pinkocto
- categories: [python]

## 화소의 개념

화소란 화면(영상)을 구성하는 가장 기본이 되는 단위를 말한다. 일반적으로 영상처리 입문에서 가장 먼저 다루는 내용이 화소값 기반 처리이다. 이것은 영상 구조에 대해 알기 위해 가장 먼저 이해해야 하는 것이 화소에 대한 기본 개념이기 때문이다.

디지털 영상은 이 화소들의 집합을 의미하며, 이 화소들에 대해 다양한 연산을 하는 것이 영상처리이다.

## 6.1 영상화소의 접근

영상처리를 아주 간단하게 말해보면, 2차원 데이터에 대한 행렬 연산이라고 할 수 있다.
따라서 영상을 다루려면 기본적으로 영상의 화소에 접근하고, 그 값을 수정하거나 새로 만들 수 있어야 한다.

### 6.1.1 화소(행렬 원소) 접근

다음은 모든 원소를 순회하여 원소값을 2배로 변경하는 예제이다.

`-` 방법1

행렬의 원소를 순회하며 직접 원소값을 가져와서 계산

In [1]:
import numpy as np

def mat_access1(mat):
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            k = mat[i, j]
            mat[i, j] = k * 2

In [3]:
mat1 = np.arange(10).reshape(2,5)
mat1

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [6]:
print('원소 처리 전: \n%s\n' % mat1)
mat_access1(mat1)
print('원소 처리 후: \n%s\n' % mat1)

원소 처리 전: 
[[ 0  2  4  6  8]
 [10 12 14 16 18]]

원소 처리 후: 
[[ 0  4  8 12 16]
 [20 24 28 32 36]]



`-` 방법2

행렬 원소를 순회하며, ndarray 클래스의 내부 메서드인 **`item()`** 함수와 **`itemset()`** 함수로 가져와서 값을 변경

In [8]:
def mat_access2(mat):
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            k = mat.item(i, j)  #
            mat.itemset((i, j), k*2)

In [10]:
mat2 = np.arange(10).reshape(2, 5)
mat2

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [11]:
print('원소 처리 전: \n%s\n' % mat2)
mat_access2(mat2)
print('원소 처리 후: \n%s\n' % mat2)

원소 처리 전: 
[[0 1 2 3 4]
 [5 6 7 8 9]]

원소 처리 후: 
[[ 0  2  4  6  8]
 [10 12 14 16 18]]



### 6.1.2  영상 반전을 수행하는 다양한 방법들

행렬을 처리하여 영상의 반전을 수행하는 다양한 방법들을 함수로 만들고, 각 방법의 수행속도를 계산해보자.

In [14]:
# Mat::ptr()을 통한 행렬 원소 접근

import numpy as np, cv2, time


## 화소 직접접근
def pixel_access1(image):
    image1 = np.zeros(image.shape[:2], image.dtype)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            pixel = image[i,j]                 # 화소접근
            image1[i, j] = 255 - pixel         # 화소할당
            
    return image1
        

In [15]:
## item() 함수
def pixel_access2(image):                         # item() 함수 접근 방법
    image2 = np.zeros(image.shape[:2], image.dtype)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            pixel = image.item(i, j)              # 화소접근
            image2.itemset((i, j), 255 - pixel)   # 화소할당
    return image2

In [20]:
## 룩업테이블
def pixel_access3(image):
    lut = [255 - i for i in range(256)]
    lut = np.array(lut, np.uint8)
    image3 = lut[image]
    return image3
    

In [61]:
## openCV
def pixel_access4(image):
    image4 = cv2.subtract(255, image)
    return image4

In [18]:
## ndarray 산술연산
def pixel_access5(image):
    image5 = 255 - image
    return image5

In [57]:
image = cv2.imread('./ghtop_images/chap06_images/bright.jpg', cv2.IMREAD_GRAYSCALE)

In [58]:
image.shape

(450, 360)

In [62]:
## 수행시간 체크 함수
def time_check(func, msg):
    start_time = time.perf_counter()
    ret_img = func(image)
    elapsed = (time.perf_counter() - start_time) * 1000
    print(msg, "수행시간 : %0.2f ms" % elapsed )
    return ret_img

In [63]:
image1 = time_check(pixel_access1, "[방법1] 직접 접근 방식")
image2 = time_check(pixel_access2, "[방법2] item() 접근 방식")
image3 = time_check(pixel_access3, "[방법3] 룩업테이블  방식")
image4 = time_check(pixel_access4, "[방법4] OpenCV 함수 방식")
image5 = time_check(pixel_access5, "[방법5] ndarray 방식")

[방법1] 직접 접근 방식 수행시간 : 550.22 ms
[방법2] item() 접근 방식 수행시간 : 108.73 ms
[방법3] 룩업테이블  방식 수행시간 : 0.92 ms
[방법4] OpenCV 함수 방식 수행시간 : 0.08 ms
[방법5] ndarray 방식 수행시간 : 0.19 ms


실행결과를 보면, OpenCV 또는 ndarray 방식으로 화소에 접근하는 경우 속도가 빠른 것을 확인할 수 있었다.

따라서 화소 직접 접근 방법보다는 OpenCV에서 제공하는 함수들을 조합하거나 ndarray 객체의 원소간 연산으로 구현 내용을 만드는 것이 좋다.

## 6.2 화소 밝기 변환

### 6.2.1 그레이 스케일 (명암도) 영상